In [1]:
pip install haystack-ai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [2]:
!pip install "sentence-transformers>=3.0.0" "huggingface_hub>=0.23.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.4 MB/s eta 0:00:00


In [3]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret
from haystack.components.generators import HuggingFaceAPIGenerator

In [4]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from pathlib import Path

In [5]:
#Preprocessing-Pipeline bauen
document_store = InMemoryDocumentStore()
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
document_joiner = DocumentJoiner()

In [6]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)

In [7]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

In [8]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

In [9]:
preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Path])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (List[Document])
  - document_cleaner.documents -> document_splitter.documents (List[Document])
  - document_splitter.documents -> document_embedder.documents (List[Document])
  - document_embedder.documents -> document_writer.documents (List[Document])

In [ ]:
#from sentence_transformers import SentenceTransformer

In [ ]:
#pip uninstall sentence-transformers

Found existing installation: sentence-transformers 3.0.1
Uninstalling sentence-transformers-3.0.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/sentence_transformers-3.0.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/sentence_transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled sentence-transformers-3.0.1


In [ ]:
#pip install "sentence-transformers>=3.0.0"

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)


In [10]:
output_dir = "/content/"  # Pfad zum Verzeichnis, wo sich die Datei befindet
data_file = Path(output_dir) / "data_na.txt"  # der Dateiname
preprocessing_pipeline.run({"file_type_router": {"sources": [data_file]}})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1157 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 36998}}

In [11]:
prompt_template = """
Given these documents, answer the question.
Documents:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""

In [12]:
retriever = InMemoryBM25Retriever(document_store=document_store)
prompt_builder = PromptBuilder(template=prompt_template)

In [13]:
#token bei Hugging-Face unter Access-Token erstellen, wichtig: write!
generator = HuggingFaceAPIGenerator(api_type="serverless_inference_api",
                                    api_params={"model": "mistralai/Mixtral-8x7B-Instruct-v0.1"},
                                    token=Secret.from_token("<add_your_api_token_here>"))

In [14]:
rag_pipeline = Pipeline()
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceAPIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [15]:
#Question 1
question = "What are all the marriages and child relationships following the final chapter? In the last chapter of Naruto, some characters got married, and some of them had kids. What is the entire husband-wife-kids set?"

In [16]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [17]:
print(results["llm"]["replies"])

[' The following characters got married and had kids:\n\n* Naruto Uzumaki and Hinata Hyuga had a son named Boruto Uzumaki.\n* Shikaku Nara and Yoshino Nara had a son named Shikamaru Nara.\n* Shikaku Nara and Yoshino Nara also had a daughter named Temari Nara.\n* Fugaku Uchiha and Mikoto Uchiha had two sons named Itachi Uchiha and Sasuke Uchiha.\n* Kurenai Yūhi and Asuma Sarutobi had a son named Mirai Sarutobi.\n* Choza Akimichi and Tsunade had a son named Chōji Akimichi.\n\nNote: The Naruto Hiden novels are not included in this timeline.']


In [18]:
#Question 2
question = "Why does Naruto use his mother's family name? Is there any logical and canonical reason to why Naruto used his mother's family name, Uzumaki, instead of his father's family name?"

In [19]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [20]:
print(results["llm"]["replies"])

[" Yes, there is a logical and canonical reason to why Naruto used his mother's family name, Uzumaki, instead of his father's family name. Naruto's full name is a reference to the Naruto whirlpools (鳴門の渦潮, Naruto no uzushio), named after the city in the Tokushima Prefecture in Japan. Additionally, Naruto's mother, Kushina Uzumaki, is a relative to the Uzumaki clan from Uzushiogakure in the Land of Whirlpools, who used the spiral pattern as both their clan and village symbol. Naruto's family name, Uzumaki, can mean 'whirlpool' or 'maelstrom', which is a recurring theme in Naruto, especially in relation to the character of Naruto Uzumaki himself."]


In [21]:
#Question 3
question = "What will happen to the tailed beast if the jinchuruki is killed? It is shown in the anime that the jinchuruki is caught and the tailed beast is extracted and sealed into the Demonic statue by the Akatsuki. What will happen if the jinchuruki is killed somewhere else? In search for Tsunade arc, during the fight between Jiraiya-Tsunade and Orochimaru, Orochimaru tries to kill Naruto as he is the jinchuruki of the nine tails. What would have happened to the nine tails if he had killed Naruto?"

In [22]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [23]:
print(results["llm"]["replies"])

[' If the jinchuriki is killed, the tailed beast will not be extracted and sealed into the Demonic statue by the Akatsuki. Instead, the tailed beast will be released and roam freely. In the scenario you mentioned, if Orochimaru had killed Naruto, the Nine-Tails would have been released and roamed freely. However, it is important to note that this is not shown in the anime or manga, so it is only speculation based on the information provided.']


In [25]:
#Question 4
question = "When did Kakashi and Obito awaken their Mangekyou Sharingan? In chapter 604 of the manga, it is shown that Kakashi kills Rin. This fulfills the requirement to activate the Mangekyou Sharingan. So that should have been the moment when Kakashi was able to use the Mangekyou Sharingan. Also, the Yellow Flash was still alive when this happens, so this means that Naruto was not yet born. This clearly points to the fact that from the very beginning of the Naruto series, Kakashi had his Mangekyou Sharingan activated. When Kakashi first fought Itachi (when Itachi and Kisame appear at Konoha) and faced the Tsukuyomi he was devastated in a moment, even though he had his Mangekyou Sharingan active (though not used). But the next time they face (during the rescue mission of Gara), he was able to withstand it. Now my first question is, was Kakashi unaware of his Mangekyou Sharingan until he faced Itachi, or was he not proficient in using it. Also why in the Naruto series (I mean the series before the start of Shippuden) didn't Kakashi use his Mangekyou Sharingan? Now for my second question, in the same chapter, a few pages back, just when Kakashi strikes the blow at Rin, Obito's vision starts to fade as he can sense Rin dying. Does this mean Obito's Mangekyou Sharingan was awakened at the same time? We know that Kakashi and Obito shares the same other dimension because of the same Sharingan that the both use (not the same eye, but the eye of the same Uchiha). So does the same apply for their activation of Mangekyou Sharingan?"

In [26]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [27]:
print(results["llm"]["replies"])

[' Kakashi and Obito both awakened their Mangekyō Sharingan after witnessing the death of Rin Nohara during the Third Shinobi World War. However, the manga does not explicitly show the moment when they activated their Mangekyō Sharingan. It is possible that Kakashi was unaware of his Mangekyō Sharingan until he faced Itachi, or he was not proficient in using it. Kakashi did not use his Mangekyō Sharingan in the Naruto series before the start of Shippuden because he was not proficient in using it. As for Obito, his vision started to fade as he sensed Rin dying, which implies that he may have awakened his Mangekyō Sharingan at the same time as Kakashi. However, the manga does not explicitly show the moment when Obito activated his Mangekyō Sharingan. It is also possible that they share the same other dimension because of the same Sharingan, but the manga does not explicitly confirm this.']


In [28]:
#Question 5
question = "What happened to the Yin half of Kurama? Did Naruto get both halves of Kurama after fighting Sasuke, or did the other half of Kurama die when Minato was killed?"

In [29]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [30]:
print(results["llm"]["replies"])

[' The Yin half of Kurama was absorbed by Black Zetsu and became its jinchūriki. Naruto did not get both halves of Kurama after fighting Sasuke. The other half of Kurama did not die when Minato was killed.']


In [31]:
#Question 6
question = "At what age did Kakashi Hatake become a Jounin? This answer on Yahoo! says he became a Jounin at the age of 13, which makes him the youngest Jounin. I have heard a lot of chatter on the subject, but I've never seen the answer stated outright (or with any proof). The youngest I've seen reference to was Kakashi. He was a chunin at 6, a jonin at 10, and joined ANBU Black Ops at 13. Itachi, on the other hand, became a chunin at 10 and became a Captain of ANBU Black Ops at 13. He's still pretty amazing, but Kakashi was younger. Is it true that he is the youngest Jounin? What is his real age when he became a Jounin? The magic number starts from at the age of 5"

In [32]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [33]:
print(results["llm"]["replies"])

[" According to the Naruto wiki, Kakashi Hatake became a Jounin at the age of 13. This makes him the youngest Jounin in Konoha's history."]


In [34]:
#Question 7
question = "Why did Kakashi's Sharingan fade after Obito died? In Chapter 691, after Obito died, Kakashi’s Sharingan faded. Why is it that Kakashi's Sharingan faded, while the Sharingan Danzo had stolen from Shisui and the one Shisui had given Itachi, were still okay even after its original owner had died?"

In [35]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [36]:
print(results["llm"]["replies"])

[" The Sharingan that Kakashi had received from Obito was a transplanted eye, while the Sharingan that Danzo had stolen from Shisui and the one Shisui had given Itachi were both implanted eyes. Transplanted eyes are not as durable as implanted eyes, and will fade over time after the original owner's death. Implanted eyes, on the other hand, are more permanent and will not fade even after the original owner's death. This is why Kakashi's Sharingan faded after Obito died, while the Sharingan that Danzo had stolen from Shisui and the one Shisui had given Itachi were still okay even after its original owner had died."]


In [37]:
#Question 8
question = "Why can't the Inuzuka clan's dogs use human language?Why can't Inuzuka clan's dogs use human language when Kakashi's dogs can? Also, how does Kiba and Akamaru understand each other?"

In [38]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [39]:
print(results["llm"]["replies"])

[" The Inuzuka clan's dogs, including Akamaru, cannot use human language. However, Kakashi's dogs, such as Pakkun, can use human language. The reason for this difference is not explicitly stated in the documents. It is possible that Kakashi's dogs have undergone some kind of special training or have a unique ability that allows them to use human language.\n\nAs for how Kiba and Akamaru understand each other, it is not explicitly stated in the documents. However, it is mentioned that Akamaru has heightened senses and abilities, and is able to work and be summoned alongside shinobi. It is possible that Kiba and Akamaru have developed a strong bond and are able to communicate with each other through body language, gestures, or other non-verbal means. Additionally, it is mentioned that Akamaru has shown the ability to use the Beast Human Clone technique, which allows him to take on a more human-like form and possibly communicate in a more human-like way. However, this technique is not cons

In [40]:
#Question 9
question = "How can Haku perform hand seals with only one hand?In the Land of the Waves arc, it can be seen that Haku performs handseals with only one hand. That must be something special, because a.) even Kakashi was surprised to see that, and b.) that's never seen again somewhere (except I forgot it).How is that possible? Is that a special technique? Was it ever explained in the manga/anime?"

In [41]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [42]:
print(results["llm"]["replies"])

[" Haku's ability to perform hand seals with only one hand is not explicitly explained in the manga or anime. It is possible that it is a unique skill that Haku possesses, but it is not mentioned as a specific technique or jutsu. It is also possible that it is simply a result of Haku's exceptional skill and proficiency in using hand seals. Regardless of the reason, it is clear that Haku's ability to perform hand seals with only one hand is a remarkable feat that sets him apart from other ninja."]


In [43]:
#Question 10
question = "Why can Naruto still use Rasenshuriken? When Naruto first used Rasenshuriken against Kakuzu the effect was massive. He destroyed two of Kakuzu's hearts, but there were a huge after effect to him. If I'm not mistaken, Tsunade advised Kakashi to ask Naruto never to use the technique again due to the huge risk. So why is Naruto still able to use the technique and have no side effects after that battle? I may have missed out on something."

In [44]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [45]:
print(results["llm"]["replies"])

[' Naruto is able to use Rasenshuriken without any side effects after the battle with Kakuzu because he learned senjutsu and infused the Rasenshuriken with senjutsu chakra. This maintains the shape of the Rasenshuriken after formation, allowing him to use it as a projectile and eliminating the risk of damaging himself.']


In [46]:
#Question 11
question = "What is the jutsu that ninjas use to appear/disappear instantly? Some users of the jutsu are Kakashi, Gai, Genma, Kabuto, etc. It can often be seen when Anbu members are dismissed."

In [47]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [48]:
print(results["llm"]["replies"])

[' The jutsu is called "Body Flicker Technique" (瞬身の術, Shunshin no Jutsu). It is a high-speed movement technique used by ninjas to move at extremely high speeds, appearing to disappear and reappear at will. The technique is often used by Anbu members when they are dismissed, as it allows them to quickly leave the area without being noticed. Kakashi, Gai, Genma, Kabuto, and many other ninjas are known to use this technique.']


In [50]:
#Question 12
question = "Why doesn't Kakashi use his copy ability that often? Kakashi is known as the copy cat ninja. The Ninjas from the other countries call Kakashi the Copy Cat Ninja because of his Sharingan and how he uses it for his advantage (Copying moves). But he does not use his power to copy very often (or at least not anymore). It is only shown around 3-4 times that he actually copies and uses it against somebody. Why does he not use this power more often?"

In [51]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [52]:
print(results["llm"]["replies"])

[" The document does not provide a clear answer as to why Kakashi does not use his copy ability more often. However, it does mention that Kakashi's use of Kamui, a technique he can use with his Sharingan to teleport himself or others, demands a great deal of chakra and can damage his vision with repeated use. It is possible that Kakashi is cautious about using his Sharingan too much due to these limitations. Additionally, the document mentions that Kakashi has his own personal touches when copying techniques, which may suggest that he prefers to use his own versions of techniques rather than relying solely on copies. However, without further information, it is difficult to say for certain why Kakashi does not use his copy ability more often."]


In [53]:
#Question 13
question = "Are people arware of who Naruto's parents are? Every villager in Konoha is aware that Naruto has the Kyuubi sealed inside of him.Is anyone in the village or elsewhere (other than Kakashi and Jiraiya, that I remember knew) aware that he is the son of Minato and Kushina? My main doubt being: do they know that he is the son of the Fourth Hokage?"

In [54]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [55]:
print(results["llm"]["replies"])

[' No, the villagers are not aware that Naruto is the son of Minato and Kushina. They only know that he has the Kyuubi sealed inside of him. The only characters that are aware of his parentage are Kakashi and Jiraiya.']


In [56]:
#Question 14
question = "Does every ninja become a jonin? In Naruto, a number of ninjas have become jonin, such as Neji and Kakashi. Does every ninja who becomes a chunin by definition become a jonin if they live long enough, or do some remain a chunin forever?"

In [57]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [58]:
print(results["llm"]["replies"])

[' Not every ninja who becomes a chunin becomes a jonin if they live long enough. In Naruto, Tsunade promoted Lee and Tenten to chunin, while Neji was promoted to jonin. This suggests that some ninjas remain a chunin forever, while others are promoted to jonin.']


In [59]:
#Question 15
question = "Can a child inheritboth their parents' clans jutsu? Can a child in Naruto inherit both abilities from his parents' clans? For example, if Ino and Sasuke had a child is there a possibility for it to have both Uchiha's Sharingan and Yamanaka's mind transfer jutsu?"

In [60]:
results = rag_pipeline.run(
    {
        "retriever": {"query": question},
        "prompt_builder": {"question": question},
    }
)

In [61]:
print(results["llm"]["replies"])

[" Yes, it is possible for a child in Naruto to inherit both abilities from their parents' clans. However, it is not explicitly stated in the documents provided whether Ino and Sasuke's child would have both the Sharingan and the Mind Transfer Jutsu. In the Naruto universe, a child can inherit abilities from both parents, but it depends on the specific abilities and how they are passed down. In the case of the Sharingan, it is a dōjutsu that can be transplanted into non-Uchiha, but they are unable to deactivate it and its use has a greater chakra demand for them than for actual Uchiha. The Mind Transfer Jutsu, on the other hand, is a technique specific to the Yamanaka clan. It is not explicitly stated whether it can be inherited or passed down to non-Yamanaka clan members. Therefore, while it is possible for a child in Naruto to inherit both abilities from their parents' clans, it is not explicitly stated whether Ino and Sasuke's child would have both the Sharingan and the Mind Transfe